# Loading Personalized Text ChatBot

The previous jupyter file went over on how to create the chatbot, but from now on we can just load our presaved model to save time.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
print(tf.__version__)

2.11.0


In [3]:
model = tf.keras.models.load_model('model.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 174)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 106)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 174, 200)     1484200     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 106, 200)     1484200     ['input_2[0][0]']                
                                                                                              

In [4]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

questions = []
answers = []
with open('texts.txt', 'r', encoding="utf-8") as f:
    for line in f:
        if line.startswith('Text'):
            questions.append(line[6:].rstrip('\n'))
        else:
            if line.startswith('Mateen'):
                answers.append(line[8:].rstrip('\n'))
                

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
print(len(questions), len(answers))
questions = questions[:10000]
answers = answers[:10000]
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

24054 24055
VOCAB SIZE : 7421


In [5]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )
print('done')

(10000, 174) 174
(10000, 106) 106
(10000, 106, 7421)
done


In [6]:
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output
encoder_states = [ state_h_enc , state_c_enc ]
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,), name='input_3')
decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs = model.input[1]
decoder_embeddings = model.layers[3](decoder_inputs)
decoder_outputs, state_h_dec, state_c_dec = model.layers[5](decoder_embeddings, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]

decoder_outputs = model.layers[6](decoder_outputs)
decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [7]:

def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [8]:
while True:
    try:
        user_input = input("Enter message: ")
        if user_input in ['Bye', 'bye', 'goodbye', 'Goodbye']:
            print('Goodbye, have a nice day!')
            break
        states_values = encoder_model.predict( str_to_tokens( user_input ) )
        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[0, 0] = tokenizer.word_index['start']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition :
            dec_outputs , h , c = decoder_model.predict([ empty_target_seq ] + states_values )
            sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
            sampled_word = None
            for word , index in tokenizer.word_index.items() :
                if sampled_word_index == index :
                    decoded_translation += ' {}'.format( word )
                    sampled_word = word

            if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            states_values = [ h , c ] 

        print( decoded_translation )
    except KeyError:
        print('I am not sure...')
        continue


Enter message: hello
1/1 [==============================] - 0s 19ms/step
 sorry end
Enter message: hi
1/1 [==============================] - 0s 20ms/step
 hi end
Enter message: heyyyy
I am not sure...
Enter message: heyy
I am not sure...
Enter message: whats poppin
1/1 [==============================] - 0s 20ms/step
 i put a ride with the first time in the boys end
Enter message: the boys
1/1 [==============================] - 0s 20ms/step
 ethan coined the end
Enter message: los altos lifestyle
1/1 [==============================] - 0s 22ms/step
 maybe sam end
Enter message: who are the boys
1/1 [==============================] - 0s 19ms/step
 mason just pick me up end
Enter message: can you pick me up
1/1 [==============================] - 0s 21ms/step
 that was rly funny as well end
Enter message: tell me a joke
1/1 [==============================] - 0s 20ms/step
 any of the guys next week and then not rly down to hang with us end


KeyboardInterrupt: Interrupted by user